In [1]:
import numpy as np

In [2]:
choices = [0, 1]
input_img = np.random.choice(choices, size=(6, 6))
print(input_img.shape)
print(repr(input_img))

(6, 6)
array([[1, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 1],
       [0, 1, 1, 0, 0, 0],
       [1, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1]])


In [3]:
arr1 = np.array([[1, 2], [3, 4]])
arr2 = np.array([[1, 2], [3, 4]])
print(repr(arr1), repr(arr2))

array([[1, 2],
       [3, 4]]) array([[1, 2],
       [3, 4]])


In [4]:
print(np.sum(arr1*arr2))

30


In [5]:
len2 = np.zeros((1, 2))
len3 = np.zeros((3))
print(repr(len2.T))
print(repr(len3))

array([[0.],
       [0.]])
array([0., 0., 0.])


In [6]:
arr3 = np.append(arr1, len2.T, axis=1)
print(repr(arr3))

array([[1., 2., 0.],
       [3., 4., 0.]])


In [20]:
def get_padded(input_: np.ndarray, filter_size: int, padding: str) -> np.ndarray:
    if padding.lower() == 'valid':
        return input_
    elif padding.lower() == 'same':
        nc, nh, nw = input_.shape
        fh, fw = filter_size, filter_size
        result = np.zeros((nc, nh + 2 * (fh//2), nw + 2 * (fw//2)))
        print(result.shape)
        result[:, fh//2: fh//2 + nh, fw//2: fw//2 + nw] += input_
        return result
    else:
        print('Invalid padding value.')
        return None

In [33]:
inp = np.ones((4, 4, 3))
res = get_padded(inp, 3, 'same')
print(repr(res))
print(repr(res.flatten()), res.flatten().shape)

(4, 6, 5)
array([[[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 0.]]])
array([0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1.,
       1., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 0.,
       1., 1., 1., 0., 0., 0., 0., 0.

In [22]:
def relu(x):
    x[x < 0] = 0
    return x

In [23]:
def get_custom_weights(n_filters, filter_height, filter_width, filter_channel):
    assert(n_filters == filter_channel)
    filters = np.zeros((n_filters, filter_channel, filter_height, filter_width))
    for i in range(n_filters):
        filters[i, i, :, :] += 1
#     print(np.count_nonzero(filters==1, axis=1))
    return filters

def get_reshaped(input_matrix):
    inp_height, inp_width, inp_channel = input_matrix.shape
    reshaped = np.zeros((inp_channel, inp_height, inp_width))
    for i in range(inp_channel):
        reshaped[i, :, :] = input_matrix[:, :, i]
    return reshaped

def convolve(inp_matrix, weights, stride):
    
    inp_channel, inp_height, inp_width = inp_matrix.shape
    n_filters, filter_channel, filter_height, filter_width = weights.shape
    
    assert(inp_channel == filter_channel)
    
    out_height = int(np.floor((inp_height - 1) // stride) + 1)
    out_width = int(np.floor((inp_width - 1) // stride) + 1)
    out_ = np.zeros((n_filters, out_height, out_width))
    
    for filter_id in range(n_filters):
        out_row = -1
        for row in range(0, inp_height, stride):
            r_start, r_end = row, row + filter_height
            if r_end > inp_height:
                break
            out_row += 1
            out_col = -1
            for col in range(0, inp_width, stride):
                c_start, c_end = col, col + filter_width
                if c_end > inp_width:
                    break
                out_col += 1
                out_[filter_id, out_row, out_col] = np.sum(weights[filter_id] * inp_matrix[:, r_start: r_end, c_start: c_end])
    return out_

def conv(inp_matrix, n_filters, padding='valid', stride=1, f_size=5, activation=relu, channels_first=True):
    if not channels_first:
        inp_matrix = get_reshaped(inp_matrix)
    print('------------------------------')
    print('Input_shape : ', inp_matrix.shape)
    inp_channel = inp_matrix.shape[0]
    weights = get_custom_weights(n_filters, f_size, f_size, inp_channel)
    padded = get_padded(inp_matrix, f_size, padding)
    convolved = convolve(padded, weights, stride)
    print('weights_shape : ', weights.shape)
    print('padded_shape : ', padded.shape)
    print('convolved_shape : ', convolved.shape)
    return convolved

def get_dense_weights(size, n_out):
    weights = np.zeros((n_out, size))
    factor = size // n_out
    for i in range(n_out):
        weights[i, i*factor: i*factor + factor] += 1
    return weights
    
def dense(input_vector, n_out):
    weights = get_dense_weights(input_vector.shape[0], n_out)
    out_ = np.zeros((n_out))
    for i in range(n_out):
        out_[i] = np.sum(weights[i] * input_vector)
    return out_

In [24]:
np.random.seed(0)
inp = np.random.choice([0, 1], size=(300, 300, 3))
resh = get_reshaped(inp)
# print(repr(inp))
print(np.count_nonzero(resh[0, :, :]==1))
print(np.count_nonzero(resh[1, :, :]==1))
print(np.count_nonzero(resh[2, :, :]==1))

44950
45126
45187


In [25]:
conv1 = conv(inp, n_filters=3, stride=5, channels_first=False)

------------------------------
Input_shape :  (3, 300, 300)
weights_shape :  (3, 3, 5, 5)
padded_shape :  (3, 300, 300)
convolved_shape :  (3, 60, 60)


In [26]:
conv2 = conv(conv1, n_filters=3, stride=5)

------------------------------
Input_shape :  (3, 60, 60)
weights_shape :  (3, 3, 5, 5)
padded_shape :  (3, 60, 60)
convolved_shape :  (3, 12, 12)


In [27]:
conv3 = conv(conv2, n_filters=3, f_size=3, stride=3)

------------------------------
Input_shape :  (3, 12, 12)
weights_shape :  (3, 3, 3, 3)
padded_shape :  (3, 12, 12)
convolved_shape :  (3, 4, 4)


In [28]:
conv4 = conv(conv3, n_filters=3, f_size=3, stride=3, padding='same')

------------------------------
Input_shape :  (3, 4, 4)
(3, 6, 6)
weights_shape :  (3, 3, 3, 3)
padded_shape :  (3, 6, 6)
convolved_shape :  (3, 2, 2)


In [29]:
flattened = conv4.flatten()
print(repr(flattened))
print(flattened.shape)

array([11252., 11255., 11227., 11216., 11290., 11240., 11288., 11308.,
       11160., 11335., 11356., 11336.])
(12,)


In [30]:
output = dense(flattened, 3)

In [31]:
print(output)

[44950. 45126. 45187.]


In [40]:
print(tuple(output))

(44950.0, 45126.0, 45187.0)


In [42]:
print(output.shape)

(3,)
